In [1]:
import pandas as pd
import numpy as np
import httpx
from tqdm import tqdm

import re

In [2]:
def build_prompt(patient_data):
    return f"""
    You are an emergency department physician. You are presented with the following patient data.
    Provide a concise summary of irregularities in the patients data. Also provide a possible diagnosis.
    Do not provide further treatment recommendations.

    patient data:
    temperature: {patient_data['temperature']}°F
    heart rate: {patient_data['heartrate']} bpm
    o2 saturation: {patient_data['o2sat']}%
    systolic blood pressure: {patient_data['sbp']} mmHg
    diastolic blood pressure: {patient_data['dbp']} mmHg
    respiratory rate: {patient_data['resprate']} bpm
    pain level: {patient_data['pain']}
    acuity level: {patient_data['acuity']}
    age on admission: {patient_data['age_on_adm']} years
    gender: {patient_data['gender']}
    transport mode: {patient_data['arrival_transport']}
    chief complaint: {patient_data['chiefcomplaint']}
    """

In [3]:
def rate_full_patient(patient_data):
    url = "http://localhost:11434/api/generate"

    with httpx.Client(timeout=600) as client:
        data_payload = {
            "model": "mistral:instruct",
            "prompt": build_prompt(patient_data),
            "stream": False,
        }

        res = client.post(url, json=data_payload)

        try:
            res.raise_for_status()
            # get response field from the json
            response = res.json()["response"]
        except Exception as e:
            print(e)
            response = ""

        return response

In [4]:
base_frame = pd.read_csv("../processed_data/mistral_full_data.csv")

/tmp/ipykernel_2230/807395881.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  base_frame = pd.read_csv("../processed_data/mistral_full_data.csv")


In [5]:
build_prompt(base_frame.iloc[0])

'\n    You are an emergency department physician. You are presented with the following patient data.\n    Provide a concise summary of irregularities in the patients data. Also provide a possible diagnosis.\n    Do not provide further treatment recommendations.\n\n    patient data:\n    temperature: 101.1°F\n    heart rate: 87.0 bpm\n    o2 saturation: 95.0%\n    systolic blood pressure: 133.0 mmHg\n    diastolic blood pressure: 63.0 mmHg\n    respiratory rate: 20.0 bpm\n    pain level: 3\n    acuity level: 3.0\n    age on admission: 81 years\n    gender: M\n    transport mode: WALK IN\n    chief complaint: ILI\n    '

In [6]:
input_columns = [
    "temperature",
    "heartrate",
    "o2sat",
    "sbp",
    "dbp",
    "resprate",
    "pain",
    "acuity",
    "age_on_adm",
    "gender",
    "arrival_transport",
    "chiefcomplaint",
]

In [7]:
rate_full_patient(base_frame.iloc[0])

" The patient is an 81-year-old male who presented to the emergency department as a walk-in with a chief complaint of an Illness (ILI). The following irregularities are noted in the provided data:\n\n1. Elevated temperature: The patient's temperature is recorded at 101.1°F (38.4°C), which is above the normal range for body temperature.\n2. Low diastolic blood pressure: The patient's diastolic blood pressure is recorded at 63.0 mmHg, which is below the normal range for this measurement in adults.\n3. Elevated respiratory rate: The patient's respiratory rate is recorded at 20.0 breaths per minute, which is higher than the normal range for adults at rest.\n\nBased on these findings, it is possible that the patient is experiencing an infection, such as pneumonia or sepsis, given the chief complaint of ILI and the fever, tachycardia (elevated heart rate), and respiratory distress (elevated respiratory rate). However, a definitive diagnosis would require further evaluation, including laborat

In [7]:
# only rate the rows that have not been rated, llm info is empty
to_rate = base_frame[base_frame["llm_info"].isnull()]

for i, row in tqdm(to_rate.iterrows(), total=to_rate.shape[0]):
    rated = rate_full_patient(row)
    base_frame.loc[i, "llm_info"] = rated

    if i % 10 == 0:
        base_frame.to_csv("../processed_data/mistral_full_data.csv", index=False)

base_frame.to_csv("../processed_data/mistral_full_data.csv", index=False)

  2%|▏         | 8173/375916 [17:14:15<775:36:08,  7.59s/it] 


KeyboardInterrupt: 

In [ ]:
base_frame.head()